In [15]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
from tensorflow.keras.utils import to_categorical

In [3]:
# load a clean dataset
def load_clean_sentences(filename):
	return load(open(filename, 'rb'))
 
# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer
 
# max sentence length
def max_length(lines):
	return max(len(line.split()) for line in lines)
 
# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
	# integer encode sequences
	X = tokenizer.texts_to_sequences(lines)
	# pad sequences with 0 values
	X = pad_sequences(X, maxlen=length, padding='post')
	return X
 
# one hot encode target sequence
def encode_output(sequences, vocab_size):
	ylist = list()
	for sequence in sequences:
		encoded = to_categorical(sequence, num_classes=vocab_size)
		ylist.append(encoded)
	y = array(ylist)
	y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
	return y
 
# define NMT model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
	model = Sequential()
	model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
	model.add(LSTM(n_units))
	model.add(RepeatVector(tar_timesteps))
	model.add(LSTM(n_units, return_sequences=True))
	model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
	return model
 


In [4]:
# load datasets
dataset = load_clean_sentences('Both.pkl')
train = load_clean_sentences('Train.pkl')
test = load_clean_sentences('Test.pkl')

In [5]:
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset.values[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset.values[:, 0])
print('English Vocabulary Size: %d' % eng_vocab_size)
print('English Max Length: %d' % (eng_length))
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset.values[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset.values[:, 1])
print('German Vocabulary Size: %d' % ger_vocab_size)
print('German Max Length: %d' % (ger_length))

English Vocabulary Size: 147330
English Max Length: 1151
German Vocabulary Size: 67815
German Max Length: 1016


In [6]:
dataset=[]

In [7]:
# prepare training data
trainX = encode_sequences(ger_tokenizer, ger_length, train.values[:, 1])

In [8]:
trainY = encode_sequences(eng_tokenizer, eng_length, train.values[:, 0])

In [9]:
train=[]

In [19]:
len(trainX)

125000

In [21]:
trainY

array([[ 181, 4317,    1, ...,    0,    0,    0],
       [ 355,   32,   10, ...,    0,    0,    0],
       [ 132, 4239, 1310, ...,    0,    0,    0],
       ...,
       [  55,   13,  120, ...,    0,    0,    0],
       [ 766,  154,  144, ...,    0,    0,    0],
       [  13,  651,  901, ...,    0,    0,    0]])

In [ ]:
trainY = encode_output(trainY, eng_vocab_size)

In [10]:
# prepare validation data
testX = encode_sequences(ger_tokenizer, ger_length, test.values[:, 1])
testY = encode_sequences(eng_tokenizer, eng_length, test.values[:, 0])
test=[]

In [ ]:
testY = encode_output(testY, eng_vocab_size)

In [11]:
# define model
model = define_model(ger_vocab_size, eng_vocab_size, ger_length, eng_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [13]:
# summarize defined model
print(model.summary())
plot_model(model, to_file='model.png', show_shapes=True)
# fit model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1016, 256)         17360640  
_________________________________________________________________
lstm (LSTM)                  (None, 256)               525312    
_________________________________________________________________
repeat_vector (RepeatVector) (None, 1151, 256)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1151, 256)         525312    
_________________________________________________________________
time_distributed (TimeDistri (None, 1151, 147330)      37863810  
Total params: 56,275,074
Trainable params: 56,275,074
Non-trainable params: 0
_________________________________________________________________
None
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz

In [22]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=6)
model.fit(trainX, trainY, epochs=30, batch_size=64, validation_data=(testX, testY), callbacks=[es,checkpoint], verbose=2)

Epoch 1/30


ValueError: in user code:

    C:\Users\Rameel\Anaconda3\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\Rameel\Anaconda3\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Rameel\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Rameel\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Rameel\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Rameel\Anaconda3\lib\site-packages\keras\engine\training.py:835 run_step  **
        outputs = model.train_step(data)
    C:\U